In [91]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, r2_score, accuracy_score
from sklearn.svm import  SVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, ElasticNet
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import warnings
warnings.simplefilter('ignore')
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, BaggingRegressor, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, GradientBoostingClassifier, StackingClassifier, RandomForestRegressor, StackingRegressor
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor, LGBMClassifier

In [92]:
train = pd.read_csv("./HRAnalytics/train_LZdllcl.csv", index_col=0)
test = pd.read_csv("./HRAnalytics/test_2umaH9m.csv", index_col=0)

In [93]:
train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [94]:
train.isnull().sum()

department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [95]:
X_train, y_train = train.drop('is_promoted', axis=1), train['is_promoted']

In [96]:
imp_const = SimpleImputer(strategy='constant', fill_value="Bachelor's")
imp_freq = SimpleImputer(strategy='most_frequent')
ct_imp = make_column_transformer((imp_const, make_column_selector(dtype_include=object)),(imp_freq, make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False).set_output(transform='pandas')
X_trn_imp = ct_imp.fit_transform(X_train)
X_tst_imp = ct_imp.transform(test)

In [97]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
ct_ohe = make_column_transformer((ohe, make_column_selector(dtype_include=object)),("passthrough", make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False).set_output(transform='pandas')

In [98]:
X_trn_ohe = ct_ohe.fit_transform(X_trn_imp)
X_tst_ohe = ct_ohe.transform(X_tst_imp)


In [99]:
X_trn_ohe.shape, X_tst_ohe.shape

((54808, 53), (23490, 53))

In [100]:
from catboost import CatBoostRegressor, CatBoostClassifier


In [101]:
# gbm = LGBMClassifier(random_state=42)
# pipe_rf = Pipeline([('IMP',ct_imp),('OHE',ct_ohe),('GBM',gbm)])
# pipe_rf.fit(X_train, y_train)
# y_pred = pipe_rf.predict(test)

In [102]:
# # CatBoost without Hot encoding
# # {'classifier__depth': 3, 'classifier__iterations': 50, 'classifier__learning_rate': 0.1}
# gbm = CatBoostClassifier(random_state=42, max_depth=3, iterations=50, learning_rate=0.1, cat_features=['department','region','education','gender','recruitment_channel'])
# pipe_rf = Pipeline([('IMP',ct_imp),('GBM',gbm)])
# pipe_rf.fit(X_train, y_train)
# y_pred = pipe_rf.predict(test)

In [103]:
submit = pd.read_csv("./HRAnalytics/sample_submission_M0L0uXE.csv")


In [104]:
submit['is_promoted'] = y_pred

In [105]:
submit.to_csv("./submitHR3.csv", index=False)


In [108]:
svm = SVC(kernel='linear')
rf = RandomForestClassifier(random_state=24)
dtc = DecisionTreeClassifier(random_state=24)
xgb = XGBClassifier(random_state=24)
stack = StackingClassifier(estimators=[('SVM', svm),('LR',rf),('TREE',dtc)], final_estimator=xgb, n_jobs=-1)
stack.fit(X_trn_ohe, y_train)
y_pred = stack.predict(X_tst_ohe)

In [109]:
submit['is_promoted'] = y_pred

In [110]:
submit.to_csv("./submitHR3.csv", index=False)